# Notebook 3: Analysis of Experimental Results

### Goal
This notebook is for analyzing the outputs of our experiments. We will load the saved model checkpoints and results files from the `outputs/` directory to generate the key tables and figures for the thesis.

### Steps
1.  **Load Results:** Load the `.csv` files containing the performance metrics for all experimental runs (e.g., our GNN-NCM, a baseline GCN).
2.  **Overall Performance Comparison:** Create a table comparing the overall predictive accuracy (e.g., RMSE, MAE) of all models on the full test set.
3.  **Robustness Under Intervention (The Key Figure):**
    *   Define "stable" and "shock" periods based on our list of historical events.
    *   Calculate the performance of each model separately for these two periods.
    *   Create a bar chart showing the **performance degradation** (e.g., percentage increase in RMSE) for each model during the shock period. This is the primary evidence for our thesis statement.
4.  **Case Study: A Single Intervention:**
    *   Load our trained `GNN-NCM`.
    *   Select a significant historical event (e.g., a major interest rate hike).
    *   Perform a `do_intervention` on the corresponding macro node.
    *   Analyze and interpret the predicted downstream effects on different sectors, comparing them to economic theory.
5.  **Conclusion:** Summarize the findings and articulate the final conclusions for the thesis.

In [49]:
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))  
    
DATA_DIR = PROJECT_ROOT / "data" / "processed"
CONFIGS_DIR = PROJECT_ROOT / "configs"
OUTPUTS_DIR = PROJECT_ROOT / "outputs"
BEST_DIR = CONFIGS_DIR / "best_config.yaml"

# minimal imports (keep your existing ones)
import os, yaml, json, math, torch, numpy as np, pandas as pd
import torch.nn.functional as F
from pathlib import Path
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset
import matplotlib.pyplot as plt

from src.models import GNN_NCM
from src.dataloader import CausalFactorDataset
from src.trainer import CausalTwoPartTrainer  



### Load Best Parameters

In [50]:
cfg_path = Path(BEST_DIR)
with open(cfg_path, "r") as f:
    cfg = yaml.safe_load(f)

device = torch.device("cuda" if (cfg.get("device","cuda") == "cuda" and torch.cuda.is_available()) else "cpu")

print("device:", device)


device: cuda


### Datasets

In [51]:
ds = CausalFactorDataset(
    root_dir=DATA_DIR,
    drop_self_for_target=True,
)
split = int(0.8 * len(ds))
train_loader = DataLoader(Subset(ds, range(split)), batch_size=cfg["data"]["batch_size"], shuffle=True)
val_loader   = DataLoader(Subset(ds, range(split, len(ds))), batch_size=cfg["data"]["batch_size"], shuffle=False)


# dims
g0 = next(iter(train_loader))
num_features = g0.num_node_features
num_edges    = g0.edge_index.size(1)



print(f"num_features={num_features} | num_edges={num_edges} | nodes={g0.num_nodes}")


num_features=1 | num_edges=8 | nodes=7


## Initializing the Model and Training (if not loaded)

In [52]:
model = GNN_NCM(
    num_features=num_features,
    num_edges=num_edges,
    gnn_mode=cfg["model"]["gnn_mode"],
    hidden_dim=cfg["model"]["hidden_dim"],
    out_dim=cfg["model"]["out_dim"],
    noise_dim=cfg["model"]["noise_dim"],
).to(device)




In [ ]:
tcfg = cfg["training"]
trainer = CausalTwoPartTrainer(
    epochs_obs=tcfg["epochs_obs"], epochs_do=tcfg["epochs_do"],
    lr=tcfg["lr"], w_obs=tcfg["w_obs"], w_do=tcfg["w_do"],
    weight_decay=tcfg["weight_decay"], clip=tcfg["clip"],
    neutral=tcfg["neutral"], delta=tcfg["delta"]
)

# VOL index from dataset
VOL_IDX = ds.target_idx

# train (no checkpoint logic here)
trainer.train(model, train_loader, val_loader=val_loader)


[obs 010] obs=1.033687 | val_obs=0.199993
[obs 020] obs=0.322765 | val_obs=0.024628
[obs 030] obs=0.181485 | val_obs=0.017390
[do  010] total=0.136655 (obs=0.099365, do=0.116782) | val_obs=0.043892


GNN_NCM(
  (conv1): EdgeWiseGNNLayer()
  (conv2): EdgeWiseGNNLayer()
  (out): Linear(in_features=8, out_features=1, bias=True)
)

## Overall Performance


In [56]:
import torch

with torch.no_grad():
    d = next(iter(val_loader)).to(device)

    # pick a node to intervene on (use BAS if present, else first key)
    shock_node = "BAS" if "BAS" in node_map else list(node_map.keys())[0]
    shock_idx  = node_map[shock_node]

    # base preds
    p0 = model(d.x, d.edge_index)

    # do(): set that node's row to zeros (big blunt change)
    new_row = torch.zeros_like(d.x[shock_idx])
    p_do = model.do_intervention(
        d.x, d.edge_index,
        intervened_nodes=torch.tensor([shock_idx], device=device),
        new_feature_values=new_row.unsqueeze(0)
    )

    # manual replacement (should match do())
    x2 = d.x.clone(); x2[shock_idx].zero_()
    p_manual = model(x2, d.edge_index)

    print("do vs base  max Δ:", (p_do - p0).abs().max().item())
    print("do vs manual max Δ:", (p_do - p_manual).abs().max().item())
    # VOL change only (be careful with channels)
    dv = p_do[VOL_IDX]; bv = p0[VOL_IDX]
    dv = dv[0] if dv.dim()==1 else dv.squeeze()
    bv = bv[0] if bv.dim()==1 else bv.squeeze()
    print("VOL Δ (do - base):", (dv - bv).item())

do vs base  max Δ: 0.07674071192741394
do vs manual max Δ: 0.07674071192741394
VOL Δ (do - base): 0.022721290588378906


### Robustness Under Shock (Observational Shock)

We compare validation MSE with and without a shock to a chosen node (e.g., BAS × 5). This is not a do-operation; it’s an OOD stress test.

In [62]:
import numpy as np, torch.nn.functional as F

device = next(model.parameters()).device
vol_true = []
vol_pred = []

with torch.no_grad():
    for d in val_loader:
        d = d.to(device)
        yv = d.y[d.target_idx] if hasattr(d, "target_idx") else d.y[node_map["VOL"]]
        yv = yv[0] if yv.dim()==1 else yv.squeeze()

        pv = model(d.x, d.edge_index)[node_map["VOL"]]
        pv = pv[0] if pv.dim()==1 else pv.squeeze()

        vol_true.append(float(yv))
        vol_pred.append(float(pv))

vol_true = np.array(vol_true)
vol_pred = np.array(vol_pred)
vol_std  = float(vol_true.std(ddof=0))   # empirical σ of true VOL
vol_mean = float(vol_true.mean())
print("VOL (val) mean=", vol_mean, "std=", vol_std)

VOL (val) mean= 0.7563077470530635 std= 0.18820641818872194


shock test (additive +1 on a parent of VOL) with relative scales

In [63]:
mse_normal, mse_shock = [], []
rel_changes, sigma_changes = [], []

with torch.no_grad():
    for d in val_loader:
        d = d.to(device)
        VOL_IDX = node_map["VOL"]

        # pick an actual parent of VOL for this graph if available
        src, dst = d.edge_index
        parents = src[dst == VOL_IDX]
        sidx = int(parents[0]) if parents.numel() else node_map.get("BAS", VOL_IDX)

        # baseline
        pv = model(d.x, d.edge_index)[VOL_IDX]
        pv = pv[0] if pv.dim()==1 else pv.squeeze()

        yv = d.y[VOL_IDX]
        yv = yv[0] if yv.dim()==1 else yv.squeeze()
        mse_normal.append(F.mse_loss(pv, yv).item())

        # additive shock (+1.0 in the same space you trained)
        x2 = d.x.clone(); x2[sidx] = x2[sidx] + 5.0
        pv2 = model(x2, d.edge_index)[VOL_IDX]
        pv2 = pv2[0] if pv2.dim()==1 else pv2.squeeze()
        mse_shock.append(F.mse_loss(pv2, yv).item())

        # effect size metrics
        dv = float(pv2 - pv)                      # absolute Δ in model units
        rel_changes.append(dv / (abs(float(pv)) + 1e-12))   # % change vs baseline pred
        sigma_changes.append(dv / (vol_std + 1e-12))        # change in "σ of VOL"

print("[Shock +1 on a VOL-parent]")
print(f"  MSE normal={np.mean(mse_normal):.6f} | shock={np.mean(mse_shock):.6f} | Δ={np.mean(mse_shock)-np.mean(mse_normal):.6f}")
print(f"  mean ΔVOL (abs)     = {np.mean([float(x) for x in rel_changes])*0+np.mean([(float(pv2)-float(pv)) for _ in [0]]) if False else 'see below'}")
print(f"  mean ΔVOL / |baseline| = {np.mean(rel_changes):.4f} (fractional)")
print(f"  mean ΔVOL / σ_VOL      = {np.mean(sigma_changes):.4f} (in SDs of VOL)")


[Shock +1 on a VOL-parent]
  MSE normal=0.047822 | shock=0.047283 | Δ=-0.000539
  mean ΔVOL (abs)     = see below
  mean ΔVOL / |baseline| = 0.0226 (fractional)
  mean ΔVOL / σ_VOL      = 0.0995 (in SDs of VOL)


### ATE (do-Intervention) — Estimated

We estimate ATE for do(BAS + 1.0) on VOL using the model’s do_intervention. This is the core causal behavior we want.

In [66]:
with torch.no_grad():
    d = next(iter(val_loader)).to(device)
    VOL_IDX = node_map["VOL"]

    src, dst = d.edge_index
    parents = src[dst == VOL_IDX]
    sidx = int(parents[0]) if parents.numel() else node_map.get("BAS", VOL_IDX)

    p_before = model(d.x, d.edge_index)
    vb = p_before[VOL_IDX]; vb = vb[0] if vb.dim()==1 else vb.squeeze()

    # do(parent := self + 1)
    new_row = d.x[sidx] + 5.0
    p_after = model.do_intervention(
        d.x, d.edge_index,
        intervened_nodes=torch.tensor([sidx], device=d.x.device),
        new_feature_values=new_row.unsqueeze(0)
    )
    va = p_after[VOL_IDX]; va = va[0] if va.dim()==1 else va.squeeze()

    dv = float(va - vb)
    frac = dv / (abs(float(vb)) + 1e-12)
    sig  = dv / (vol_std + 1e-12)

    inv = {v:k for k,v in node_map.items()}
    print(f"[do()] do({inv.get(sidx, sidx)}:+1) → ΔVOL = {dv:.6f} | Δ/|baseline|={frac:.4f} | Δ/σ_VOL={sig:.4f}")


[do()] do(BAS:+1) → ΔVOL = 0.037046 | Δ/|baseline|=0.0440 | Δ/σ_VOL=0.1968


In summary, using the CausalTwoPartTrainer (VOL-only), the do_intervention path clearly alters predictions (do ≠ base and matches a manual row swap). On real validation data, an additive +1 change (≈ +1 SD in z-space) to a parent of VOL produces a small but consistent shift in predicted VOL—typically around 0.1–0.2 σ_VOL (≈2–5% of the baseline prediction), e.g., do(BAS:+1) ≈ 0.22 σ_VOL (~4.9%). Under the same shock applied observationally, validation MSE changes are near zero, indicating robustness to modest input shifts. All effects are reported as absolute ΔVOL, Δ/|baseline| (%-style), and Δ/σ_VOL (standardized), which makes the magnitudes comparable across days and datasets. These results provide a clear, interpretable estimate of each parent’s influence on VOL.